In [ ]:
!pip install uv && uv pip install sovai['full'] --system > output.log 2>&1

## Anomaly Computation

Global anomalies look at the entire dataset, local anomalies focus on neighborhoods, and cluster anomalies consider the multi-dimensional structure of the data.

In [3]:
import sovai as sov

sov.token_auth(token="visit https://sov.ai/profile for your token")

### Multivariate Anomaly Detection - Accounting Factors

In [11]:
# Load ratios - takes around 
df_factors = sov.data("factors/accounting", purge_cache=True, start_date="2021"); df_factors.head()

Loading partitions: 100%|██████████| 1594/1594 [00:21<00:00, 72.54partition/s]


profitability  value  solvency  cash_flow  illiquidity  \
ticker date                                                                 
A      2021-01-01         70.000  7.000    54.000     84.000       85.000   
       2021-01-08         70.000  7.000    54.000     84.000       93.000   
       2021-01-15         70.000  7.000    55.000     84.000       80.000   
       2021-01-22         70.000  7.000    55.000     84.000       96.000   
       2021-01-29         70.000  8.000    55.000     84.000       95.000   

                   momentum_long_term  momentum_medium_term  \
ticker date                                                   
A      2021-01-01              90.000                87.000   
       2021-01-08              83.000                54.000   
       2021-01-15              79.000                48.000   
       2021-01-22              73.000                48.000   
       2021-01-29              71.000                44.000   

                   short_term_reversal  price_volatility  dividend_yield  \
ticker date                                                                
A      2021-01-01               50.000            12.000          12.000   
       2021-01-08               29.000            15.000          15.000   
       2021-01-15               47.000            13.000          13.000   
       2021-01-22               55.000            12.000          12.000   
       2021-01-29               65.000            14.000          14.000   

                   earnings_consistency  small_size  low_growth  \
ticker date                                                       
A      2021-01-01                89.000       7.000      54.000   
       2021-01-08                83.000       6.000      54.000   
       2021-01-15                90.000       6.000      54.000   
       2021-01-22                79.000       6.000      54.000   
       2021-01-29                79.000       6.000      54.000   

                   low_equity_issuance  bounce_dip  accrual_growth  \
ticker date                                                          
A      2021-01-01               93.000      69.000          69.000   
       2021-01-08               95.000      65.000          69.000   
       2021-01-15               96.000      67.000          69.000   
       2021-01-22               96.000      72.000          70.000   
       2021-01-29               96.000      81.000          70.000   

                   low_depreciation_growth  current_liquidity  low_rnd  \
ticker date                                                              
A      2021-01-01                   42.000             79.000   36.000   
       2021-01-08                   42.000             79.000   36.000   
       2021-01-15                   42.000             79.000   36.000   
       2021-01-22                   42.000             80.000   36.000   
       2021-01-29                   42.000             80.000   35.000   

                   momentum  
ticker date                  
A      2021-01-01    91.000  
       2021-01-08    73.000  
       2021-01-15    67.000  
       2021-01-22    64.000  
       2021-01-29    59.000

#### Local, Global, and Cluster Scores (For NVDA)

I only want to look at the last **three years**, the data should be in **percentile format**

In [12]:
import pandas as pd

ticker = "TSLA"

df_last_3_years = df_factors.loc[(slice(None), slice(pd.Timestamp.now() - pd.DateOffset(years=3), None)), :]

df_last_3_years = df_last_3_years.percentile()

df_anomaly_scores = df_last_3_years.anomalies("scores",ticker)

Optimal k for global anomaly detection: 49
LOF scores shape: (7750,)
Isolation Forest scores shape: (7750,)


In [13]:
df_anomaly_scores

global_anomaly_score  local_anomaly_score  \
ticker date                                                    
CELH   2022-05-13                 0.557                0.910   
       2022-05-20                 0.639                0.876   
       2022-05-27                 0.660                0.990   
       2022-06-03                 0.478                0.879   
       2022-06-10                 0.474                0.864   
...                                 ...                  ...   
ZS     2025-04-04                 0.216                0.362   
       2025-04-11                 0.183                0.387   
       2025-04-18                 0.210                0.400   
       2025-04-25                 0.213                0.357   
       2025-05-02                 0.187                0.409   

                   clustered_anomaly_score  
ticker date                                 
CELH   2022-05-13                    0.895  
       2022-05-20                    1.000  
       2022-05-27                    1.000  
       2022-06-03                    0.927  
       2022-06-10                    0.737  
...                                    ...  
ZS     2025-04-04                    0.355  
       2025-04-11                    0.366  
       2025-04-18                    0.345  
       2025-04-25                    0.351  
       2025-05-02                    0.387  

[7750 rows x 3 columns]

In [14]:
df_anomaly_scores.plot_line("local_anomaly_score", n=50)

Plotting for random tickers. Specify tickers to plot specific data.


In [15]:
df_anomaly_scores.query("ticker ==@ticker").reset_index().set_index(["date"]).drop(columns=['ticker']).plot()

In [16]:
import plotly.express as px

px.area(df_anomaly_scores.query("ticker == 'NVDA'").reset_index().set_index("date").drop(columns=['ticker']), title="NVDA Anomaly Scores Over Time", labels={"value": "Anomaly Score", "variable": "Feature"}, line_shape="spline").show()

In [17]:
import plotly.express as px

px.bar(df_anomaly_scores.query("ticker == 'NVDA'").reset_index().melt(id_vars=['date', 'ticker'], var_name='Feature', value_name='Anomaly Score'), x='date', y='Anomaly Score', color='Feature', title="NVDA Anomaly Scores Over Time", barmode='stack').show()

In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

def create_anomaly_dashboard(df_anomaly_scores, ticker):
    # Filter for the specific ticker and calculate percentile ranks
    df_ticker = df_anomaly_scores.loc[ticker].reset_index()
    df_all = df_anomaly_scores.reset_index()
    
    for score in ['global_anomaly_score', 'local_anomaly_score', 'clustered_anomaly_score']:
        df_ticker[f'{score}_percentile'] = df_ticker[score].rank(pct=True)
    
    # Calculate ratios and interactions
    for df in [df_ticker, df_all]:
        df['global_local_ratio'] = df['global_anomaly_score'] / df['local_anomaly_score']
        df['global_clustered_ratio'] = df['global_anomaly_score'] / df['clustered_anomaly_score']
        df['local_clustered_ratio'] = df['local_anomaly_score'] / df['clustered_anomaly_score']
        df['interaction_score'] = df['global_anomaly_score'] * df['local_anomaly_score'] * df['clustered_anomaly_score']
    
    # Create subplots
    fig = make_subplots(rows=3, cols=2, 
                        subplot_titles=('Anomaly Scores Over Time', 'Anomaly Score Percentiles',
                                        'Anomaly Score Ratios', 'Interaction Score',
                                        'Global vs Local Anomaly', 'Global vs Clustered Anomaly'))
    
    # Anomaly Scores Over Time
    for score, color in zip(['global_anomaly_score', 'local_anomaly_score', 'clustered_anomaly_score'], ['blue', 'red', 'green']):
        fig.add_trace(go.Scatter(x=df_ticker['date'], y=df_ticker[score], name=score.split('_')[0].capitalize(), line=dict(color=color)), row=1, col=1)
    
    # Anomaly Score Percentiles
    for score, color in zip(['global_anomaly_score', 'local_anomaly_score', 'clustered_anomaly_score'], ['blue', 'red', 'green']):
        fig.add_trace(go.Scatter(x=df_ticker['date'], y=df_ticker[f'{score}_percentile'], name=f'{score.split("_")[0].capitalize()} Percentile', line=dict(color=color)), row=1, col=2)
    
    # Anomaly Score Ratios
    for ratio, color in zip(['global_local_ratio', 'global_clustered_ratio', 'local_clustered_ratio'], ['purple', 'orange', 'brown']):
        fig.add_trace(go.Scatter(x=df_ticker['date'], y=df_ticker[ratio], name=ratio.replace('_', '/'), line=dict(color=color)), row=2, col=1)
    
    # Interaction Score
    fig.add_trace(go.Scatter(x=df_ticker['date'], y=df_ticker['interaction_score'], name='Interaction', line=dict(color='pink')), row=2, col=2)
    
    # Global vs Local Anomaly
    fig.add_trace(go.Scatter(x=df_all['global_anomaly_score'], y=df_all['local_anomaly_score'], mode='markers', name='All Tickers', marker=dict(color='lightgrey', size=5)), row=3, col=1)
    fig.add_trace(go.Scatter(x=df_ticker['global_anomaly_score'], y=df_ticker['local_anomaly_score'], mode='markers', name=ticker, marker=dict(color='red', size=8)), row=3, col=1)
    
    # Global vs Clustered Anomaly
    fig.add_trace(go.Scatter(x=df_all['global_anomaly_score'], y=df_all['clustered_anomaly_score'], mode='markers', name='All Tickers', marker=dict(color='lightgrey', size=5)), row=3, col=2)
    fig.add_trace(go.Scatter(x=df_ticker['global_anomaly_score'], y=df_ticker['clustered_anomaly_score'], mode='markers', name=ticker, marker=dict(color='red', size=8)), row=3, col=2)
    
    # Update layout
    fig.update_layout(height=1200, title_text=f"Anomaly Score Analysis Dashboard for {ticker}")
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=2)
    fig.update_xaxes(title_text="Global Anomaly Score", row=3, col=1)
    fig.update_xaxes(title_text="Global Anomaly Score", row=3, col=2)
    fig.update_yaxes(title_text="Anomaly Score", row=1, col=1)
    fig.update_yaxes(title_text="Percentile", row=1, col=2)
    fig.update_yaxes(title_text="Ratio", row=2, col=1)
    fig.update_yaxes(title_text="Interaction Score", row=2, col=2)
    fig.update_yaxes(title_text="Local Anomaly Score", row=3, col=1)
    fig.update_yaxes(title_text="Clustered Anomaly Score", row=3, col=2)
    
    return fig

# Usage
ticker = "NVDA"  # or any other ticker in your dataset
fig = create_anomaly_dashboard(df_anomaly_scores, ticker)
fig.show()

### Local, Global, and Cluster Anomalies (Feature-Level)

Here we can also analyse at a feature level what causes the anomalies for the security-date combinations.

In [19]:
def most_anomalous_features(df, ticker, years=3):
    """
    Identify and sort the most anomalous features for a given ticker.
    
    Parameters:
    df (DataFrame): The input DataFrame containing all data.
    ticker (str): The ticker symbol to analyze.
    years (int): The number of recent years to consider (default is 3).
    
    Returns:
    DataFrame: Sorted anomalous features for the latest date of the specified ticker.
    """
    # Get data for the last 3 years and calculate local anomalies

    # Filter data for the specified ticker
    ticker_data = df.query("ticker == @ticker")
    
    # Get the latest date
    max_date = ticker_data.index.max()
    
    # Get data for the latest date and transpose
    latest_data = ticker_data.query('index == @max_date').T
    
    # Sort the features based on their anomaly scores
    if latest_data.columns.nlevels > 1:
        # For multi-level columns
        first_column = latest_data.columns[0]
    else:
        # For single-level columns
        first_column = latest_data.columns[0]
    
    sorted_result = latest_data.sort_values(by=first_column, ascending=False)
    
    return sorted_result


In [20]:
# Local Most Anomalous Features
df_local = df_last_3_years.anomalies("local", ticker=ticker); df_local.head()

profitability  value  solvency  cash_flow  illiquidity  \
ticker date                                                                 
AXON   2022-05-13          0.439  0.377     0.719      0.701        0.677   
       2022-05-20          0.349  0.368     0.534      0.372        0.650   
       2022-05-27          0.348  0.356     0.721      0.452        0.483   
       2022-06-03          0.340  0.330     0.455      0.373        0.523   
       2022-06-10          0.340  0.330     0.495      0.373        0.578   

                   momentum_long_term  momentum_medium_term  \
ticker date                                                   
AXON   2022-05-13               0.330                 0.272   
       2022-05-20               0.348                 0.564   
       2022-05-27               0.369                 0.699   
       2022-06-03               0.304                 0.645   
       2022-06-10               0.445                 0.708   

                   short_term_reversal  price_volatility  dividend_yield  \
ticker date                                                                
AXON   2022-05-13                0.496             0.458           0.458   
       2022-05-20                0.719             0.354           0.354   
       2022-05-27                0.403             0.354           0.354   
       2022-06-03                0.367             0.322           0.322   
       2022-06-10                0.414             0.289           0.289   

                   earnings_consistency  small_size  low_growth  \
ticker date                                                       
AXON   2022-05-13                 0.354       0.831       0.417   
       2022-05-20                 0.375       0.536       0.438   
       2022-05-27                 0.464       0.566       0.502   
       2022-06-03                 0.322       0.413       0.515   
       2022-06-10                 0.315       0.466       0.524   

                   low_equity_issuance  bounce_dip  accrual_growth  \
ticker date                                                          
AXON   2022-05-13                0.670       0.732           0.680   
       2022-05-20                0.637       0.663           0.679   
       2022-05-27                0.641       0.491           0.679   
       2022-06-03                0.641       0.549           0.680   
       2022-06-10                0.641       0.648           0.470   

                   low_depreciation_growth  current_liquidity  low_rnd  \
ticker date                                                              
AXON   2022-05-13                    0.388              0.410    0.677   
       2022-05-20                    0.676              0.457    0.587   
       2022-05-27                    0.384              0.825    0.601   
       2022-06-03                    0.419              0.465    0.688   
       2022-06-10                    0.675              0.465    0.712   

                   momentum  anomaly_score  
ticker date                                 
AXON   2022-05-13     0.297          0.519  
       2022-05-20     0.324          0.499  
       2022-05-27     0.600          0.515  
       2022-06-03     0.365          0.452  
       2022-06-10     0.636          0.491

In [21]:
result = most_anomalous_features(df_local, ticker='NVDA'); result

ticker,NVDA
date,2025-05-02
low_growth,1.000
momentum_medium_term,0.690
earnings_consistency,0.647
price_volatility,0.629
dividend_yield,0.629
value,0.627
illiquidity,0.584
bounce_dip,0.542
low_depreciation_growth,0.513


In [22]:
## Global Most Anomalous Features
df_global = df_last_3_years.anomalies("global", ticker=ticker); df_global.head()

profitability  value  solvency  cash_flow  illiquidity  \
ticker date                                                                 
AXON   2022-05-13          0.956  0.500     0.261      0.756        0.356   
       2022-05-20          0.483  0.500     0.673      0.758        0.578   
       2022-05-27          0.602  0.500     0.403      0.293        0.505   
       2022-06-03          0.980  0.500     0.891      0.980        0.348   
       2022-06-10          0.980  0.500     0.532      0.980        0.825   

                   momentum_long_term  momentum_medium_term  \
ticker date                                                   
AXON   2022-05-13               0.415                 0.312   
       2022-05-20               0.766                 0.349   
       2022-05-27               0.319                 0.692   
       2022-06-03               0.327                 0.375   
       2022-06-10               0.766                 0.601   

                   short_term_reversal  price_volatility  dividend_yield  \
ticker date                                                                
AXON   2022-05-13                0.351             0.939           0.939   
       2022-05-20                0.474             0.811           0.811   
       2022-05-27                0.851             0.979           0.979   
       2022-06-03                0.975             0.159           0.159   
       2022-06-10                0.820             0.979           0.979   

                   earnings_consistency  small_size  low_growth  \
ticker date                                                       
AXON   2022-05-13                 0.491       0.774       0.851   
       2022-05-20                 0.325       0.496       0.313   
       2022-05-27                 0.868       0.392       0.209   
       2022-06-03                 0.284       0.946       0.432   
       2022-06-10                 0.346       0.823       0.642   

                   low_equity_issuance  bounce_dip  accrual_growth  \
ticker date                                                          
AXON   2022-05-13                0.520       0.450           0.957   
       2022-05-20                0.454       0.328           0.965   
       2022-05-27                0.767       0.532           0.964   
       2022-06-03                0.849       0.780           0.959   
       2022-06-10                0.849       0.695           0.393   

                   low_depreciation_growth  current_liquidity  low_rnd  \
ticker date                                                              
AXON   2022-05-13                    0.463              0.586    0.729   
       2022-05-20                    0.505              0.470    0.410   
       2022-05-27                    0.653              0.848    0.452   
       2022-06-03                    0.526              0.923    0.813   
       2022-06-10                    0.816              0.923    0.515   

                   momentum  anomaly_score  
ticker date                                 
AXON   2022-05-13     0.933          0.627  
       2022-05-20     0.788          0.563  
       2022-05-27     0.572          0.619  
       2022-06-03     0.743          0.647  
       2022-06-10     0.306          0.714

In [23]:
result = most_anomalous_features(df_global, ticker='NVDA'); result

ticker,NVDA
date,2025-05-02
low_equity_issuance,0.980
cash_flow,0.889
low_depreciation_growth,0.886
solvency,0.653
accrual_growth,0.647
profitability,0.606
short_term_reversal,0.520
anomaly_score,0.513
value,0.500


In [25]:
df_last_3_years

profitability  value  solvency  cash_flow  illiquidity  \
ticker date                                                                 
A      2022-05-13          0.745  0.035     0.655      0.805        0.853   
       2022-05-20          0.735  0.035     0.655      0.805        0.878   
       2022-05-27          0.735  0.035     0.655      0.805        0.821   
       2022-06-03          0.735  0.035     0.665      0.825        0.758   
       2022-06-10          0.745  0.035     0.665      0.835        0.779   
...                          ...    ...       ...        ...          ...   
ZYXI   2025-04-04          0.395  0.455     0.465      0.515        0.445   
       2025-04-11          0.395  0.455     0.465      0.515        0.674   
       2025-04-18          0.395  0.455     0.465      0.515        0.265   
       2025-04-25          0.395  0.425     0.465      0.525        0.285   
       2025-05-02          0.115  0.765     0.605      0.165        0.265   

                   momentum_long_term  momentum_medium_term  \
ticker date                                                   
A      2022-05-13               0.465                 0.670   
       2022-05-20               0.436                 0.425   
       2022-05-27               0.406                 0.314   
       2022-06-03               0.475                 0.557   
       2022-06-10               0.386                 0.244   
...                               ...                   ...   
ZYXI   2025-04-04               0.273                 0.243   
       2025-04-11               0.254                 0.234   
       2025-04-18               0.234                 0.204   
       2025-04-25               0.274                 0.374   
       2025-05-02               0.325                 0.545   

                   short_term_reversal  price_volatility  dividend_yield  \
ticker date                                                                
A      2022-05-13                0.415             0.276           0.276   
       2022-05-20                0.095             0.305           0.305   
       2022-05-27                0.175             0.275           0.275   
       2022-06-03                0.345             0.275           0.275   
       2022-06-10                0.665             0.295           0.295   
...                                ...               ...             ...   
ZYXI   2025-04-04                0.785             0.795           0.795   
       2025-04-11                0.685             0.795           0.795   
       2025-04-18                0.725             0.785           0.785   
       2025-04-25                0.305             0.785           0.785   
       2025-05-02                0.765             0.785           0.785   

                   earnings_consistency  small_size  low_growth  \
ticker date                                                       
A      2022-05-13                 0.564       0.055       0.383   
       2022-05-20                 0.213       0.055       0.383   
       2022-05-27                 0.494       0.055       0.393   
       2022-06-03                 0.544       0.055       0.393   
       2022-06-10                 0.524       0.055       0.393   
...                                 ...         ...         ...   
ZYXI   2025-04-04                 0.154       0.755       0.576   
       2025-04-11                 0.074       0.755       0.576   
       2025-04-18                 0.256       0.755       0.575   
       2025-04-25                 0.195       0.745       0.455   
       2025-05-02                 0.133       0.775       0.535   

                   low_equity_issuance  bounce_dip  accrual_growth  \
ticker date                                                          
A      2022-05-13                0.895       0.839           0.844   
       2022-05-20                0.895       0.787           0.844   
       2022-05-27                0.895       0.768      

In [26]:
## Clustered Most Anomalous Features
df_cluster = df_last_3_years.anomalies("cluster", ticker=ticker); df_cluster.head()

Estimated rank: [10, 5, 5]


profitability  value  solvency  cash_flow  illiquidity  \
ticker date                                                                 
AXON   2022-05-13          0.290  0.088     0.143      0.152        0.580   
       2022-05-20          0.313  0.091     0.048      0.174        0.240   
       2022-05-27          0.297  0.082     0.045      0.169        0.437   
       2022-06-03          0.309  0.085     0.074      0.183        0.544   
       2022-06-10          0.318  0.090     0.185      0.189        0.703   

                   momentum_long_term  momentum_medium_term  \
ticker date                                                   
AXON   2022-05-13               0.267                 0.043   
       2022-05-20               0.234                 0.215   
       2022-05-27               0.224                 0.380   
       2022-06-03               0.226                 0.256   
       2022-06-10               0.207                 0.385   

                   short_term_reversal  price_volatility  dividend_yield  \
ticker date                                                                
AXON   2022-05-13                0.264             0.080           0.080   
       2022-05-20                0.202             0.068           0.068   
       2022-05-27                0.010             0.005           0.005   
       2022-06-03                0.044             0.024           0.024   
       2022-06-10                0.548             0.037           0.037   

                   earnings_consistency  small_size  low_growth  \
ticker date                                                       
AXON   2022-05-13                 0.253       0.166       0.164   
       2022-05-20                 0.339       0.122       0.160   
       2022-05-27                 0.282       0.174       0.171   
       2022-06-03                 0.264       0.125       0.174   
       2022-06-10                 0.229       0.172       0.179   

                   low_equity_issuance  bounce_dip  accrual_growth  \
ticker date                                                          
AXON   2022-05-13                0.394       0.420           0.802   
       2022-05-20                0.373       0.480           0.799   
       2022-05-27                0.387       0.511           0.807   
       2022-06-03                0.411       0.557           0.811   
       2022-06-10                0.444       0.516           0.486   

                   low_depreciation_growth  current_liquidity  low_rnd  \
ticker date                                                              
AXON   2022-05-13                    0.094              0.279    0.262   
       2022-05-20                    0.084              0.281    0.229   
       2022-05-27                    0.071              0.266    0.251   
       2022-06-03                    0.036              0.270    0.270   
       2022-06-10                    0.019              0.267    0.268   

                   momentum  outlier_score  
ticker date                                 
AXON   2022-05-13     0.172          0.250  
       2022-05-20     0.283          0.240  
       2022-05-27     0.142          0.236  
       2022-06-03     0.277          0.248  
       2022-06-10     0.334          0.281

In [27]:
result = most_anomalous_features(df_cluster, ticker='NVDA'); result

ticker,NVDA
date,2025-05-02
small_size,0.612
illiquidity,0.532
low_depreciation_growth,0.506
dividend_yield,0.505
price_volatility,0.505
momentum_medium_term,0.472
momentum,0.461
low_growth,0.418
low_equity_issuance,0.404


In [28]:
df_full = (df_local + df_global + df_cluster)/3

In [29]:
result = most_anomalous_features(df_full, ticker='NVDA'); result

ticker,NVDA
date,2025-05-02
low_growth,0.639
low_depreciation_growth,0.635
low_equity_issuance,0.630
price_volatility,0.540
dividend_yield,0.540
momentum_medium_term,0.531
cash_flow,0.497
illiquidity,0.466
earnings_consistency,0.451


In [30]:
df_full[[result.reset_index()["index"].values[0][0]]].query("ticker == @ticker").plot_line()

Plotting for random tickers. Specify tickers to plot specific data.


In [31]:
df_last_3_years[[result.reset_index()["index"].values[0][0]]].query("ticker == @ticker").plot_line()

Plotting for random tickers. Specify tickers to plot specific data.


### Reconstruction Anomaly

The positive and negative reconstruction error can give us the direction of the anomaly

In [32]:
df_recons = df_factors.anomalies("reconstruction", "TSLA")

Data is not in percentile form. Applying percentile transformation.
Estimated rank: [15, 9, 7]


In [33]:
df_recons.head()

profitability  value  solvency  cash_flow  illiquidity  \
ticker date                                                                 
CRWD   2021-01-01         -0.013  0.120    -0.134      0.199        0.054   
       2021-01-08          0.015  0.102    -0.148      0.240        0.129   
       2021-01-15          0.014  0.111    -0.143      0.233        0.049   
       2021-01-22          0.016  0.115    -0.154      0.238       -0.171   
       2021-01-29          0.007  0.114    -0.175      0.213        0.028   

                   momentum_long_term  momentum_medium_term  \
ticker date                                                   
CRWD   2021-01-01              -0.191                -0.060   
       2021-01-08              -0.040                -0.448   
       2021-01-15              -0.048                -0.033   
       2021-01-22              -0.051                -0.050   
       2021-01-29              -0.022                 0.161   

                   short_term_reversal  price_volatility  dividend_yield  \
ticker date                                                                
CRWD   2021-01-01               -0.157             0.032           0.032   
       2021-01-08               -0.216            -0.023          -0.023   
       2021-01-15                0.141            -0.030          -0.030   
       2021-01-22               -0.068            -0.053          -0.053   
       2021-01-29                0.036            -0.077          -0.077   

                   earnings_consistency  small_size  low_growth  \
ticker date                                                       
CRWD   2021-01-01                -0.086      -0.074      -0.069   
       2021-01-08                -0.096      -0.079      -0.058   
       2021-01-15                -0.059      -0.083      -0.060   
       2021-01-22                 0.019      -0.078      -0.055   
       2021-01-29                -0.056      -0.073      -0.052   

                   low_equity_issuance  bounce_dip  accrual_growth  \
ticker date                                                          
CRWD   2021-01-01               -0.115       0.224           0.210   
       2021-01-08               -0.107       0.035           0.228   
       2021-01-15               -0.135      -0.105           0.214   
       2021-01-22               -0.208      -0.003           0.217   
       2021-01-29               -0.146       0.070           0.199   

                   low_depreciation_growth  current_liquidity  low_rnd  \
ticker date                                                              
CRWD   2021-01-01                    0.030              0.036   -0.210   
       2021-01-08                    0.031              0.036   -0.161   
       2021-01-15                    0.030              0.033   -0.172   
       2021-01-22                    0.026              0.044   -0.164   
       2021-01-29                    0.006              0.017   -0.165   

                   momentum  
ticker date                  
CRWD   2021-01-01    -0.139  
       2021-01-08    -0.245  
       2021-01-15    -0.020  
       2021-01-22    -0.041  
       2021-01-29     0.080